# Wrapping up our CNN

First of all, don't worry: this is meant to take a while!

Also we cover software engineering because we think that data scientists should be good software engineers.

Today we're going to start to move from a minimal training loop to something that is SoTA on ImageNet, things we'll cover:

- Cuda
- Convolutions
- Hooks
- Normalization
- Transforms
- Data Blocks
- Label Smoothing
- Optimization
- Weight Decay
- Skip Connection Architectures

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Callbacks

### Callbacks as GUI events

In [7]:
import ipywidgets as widgets

In [8]:
def f(o):
    print('hi')

In [9]:
w = widgets.Button(description='Click Me')

In [10]:
w

Button(description='Click Me', style=ButtonStyle())

hi
hi


In [11]:
w.on_click(f)

Meaning, when a click event occurs, **callback** to the function f.

### Creating Your Own Callback

In [12]:
from time import sleep

In [13]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [14]:
slow_calculation()

30

Imagine yourself training a deep learning model, you really want to know how it's going. We simulated slow loops by writing `slow_calculation()` and now we're going to inject a call back to check how it's doing while running:

In [15]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb:
            cb(i)
    return res

In [16]:
def show_progress(epoch):
    print(f"Awesome! We've finished epoch {epoch}!")

In [17]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

### Lambdas & Partials

In [18]:
slow_calculation(lambda epoch: print(f"Awesome! We've finished epoch {epoch}!"))

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

In [20]:
def show_progress(exclamation, epoch):
    print(f"{exclamation} We've finished epoch {epoch}!")

In [21]:
slow_calculation(lambda epoch: show_progress("OK I guess", epoch))

OK I guess We've finished epoch 0!
OK I guess We've finished epoch 1!
OK I guess We've finished epoch 2!
OK I guess We've finished epoch 3!
OK I guess We've finished epoch 4!


30

In [25]:
def make_show_progress(exclamation):
    _inner = lambda epoch: print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner

In [28]:
slow_calculation(make_show_progress("Nice"))

Nice! We've finished epoch 0!
Nice! We've finished epoch 1!
Nice! We've finished epoch 2!
Nice! We've finished epoch 3!
Nice! We've finished epoch 4!


30

You might see it done like this instead:

In [29]:
def make_show_progress(exclamation):
    def _inner(epoch):
        print(f"{exclamation}! We've finished epoch {epoch}!")
    return _inner

In [30]:
slow_calculation(make_show_progress("Nice"))

Nice! We've finished epoch 0!
Nice! We've finished epoch 1!
Nice! We've finished epoch 2!
Nice! We've finished epoch 3!
Nice! We've finished epoch 4!


30

In [31]:
slow_calculation(make_show_progress("Amazing"))

Amazing! We've finished epoch 0!
Amazing! We've finished epoch 1!
Amazing! We've finished epoch 2!
Amazing! We've finished epoch 3!
Amazing! We've finished epoch 4!


30

Because this feature is used soo much, python has `partials`:

In [33]:
from functools import partial

In [34]:
slow_calculation(partial(show_progress, "OK I guess"))

OK I guess We've finished epoch 0!
OK I guess We've finished epoch 1!
OK I guess We've finished epoch 2!
OK I guess We've finished epoch 3!
OK I guess We've finished epoch 4!


30

In [35]:
f2 = partial(show_progress, "OK I guess")

In [36]:
slow_calculation(f2)

OK I guess We've finished epoch 0!
OK I guess We've finished epoch 1!
OK I guess We've finished epoch 2!
OK I guess We've finished epoch 3!
OK I guess We've finished epoch 4!


30

### Callbacks as Callable Classes

In [37]:
class ProgressShowingCallback():
    def __init__(self, exclamation="Awesome"):
        self.exclamation = exclamation
    
    def __call__(self, epoch):
        print(f"{self.exclamation}! We've finished epoch {epoch}!")

In [38]:
cb = ProgressShowingCallback("Just Super")

In [39]:
slow_calculation(cb)

Just Super! We've finished epoch 0!
Just Super! We've finished epoch 1!
Just Super! We've finished epoch 2!
Just Super! We've finished epoch 3!
Just Super! We've finished epoch 4!


30

### Multiple callback funcs; `*args` and `**kwargs`

In [40]:
def f(*args, **kwargs):
    print(f"args: {args}; kwargs: {kwargs}")

In [41]:
f(3, 'a', thing1='Hello')

args: (3, 'a'); kwargs: {'thing1': 'Hello'}


In [42]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb:
            cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb:
            cb.after_calc(i, val=res)
    return res

Let's create a callback class for the function:

In [43]:
class PrintStepCallBack():
    def __init__(self):
        pass
    
    def before_calc(self, *args, **kwargs):
        print(f"About to start")
    
    def after_calc(self, *args, **kwargs):
        print(f"Done step")

In [44]:
slow_calculation(PrintStepCallBack())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In the previous example we didn't get an error because we used `*args` and `**kwargs`, effectively ignoring the passed parameters.

Let's now use them:

In [45]:
class PrintStatusCallBack():
    def __init__(self):
        pass
    
    def before_calc(self, epoch, **kwargs):
        print(f"About to start: {epoch}")
    
    def after_calc(self, epoch, val, **kwargs):
        print(f"Done step {epoch}: {val}")

In [46]:
slow_calculation(PrintStatusCallBack())

About to start: 0
Done step 0: 0
About to start: 1
Done step 1: 1
About to start: 2
Done step 2: 5
About to start: 3
Done step 3: 14
About to start: 4
Done step 4: 30


30

`**kwargs` are kept because we might add other parameters in the future and we don't want to break if the user added non-existing parameters, it makes the class more resilient.

### Modifying Behavior

In [56]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb, "before_calc"):
            cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb and hasattr(cb, "after_calc"):
            if cb.after_calc(i, val=res):
                print("Stopping early")
                break
    return res

In [67]:
class PrintAfterCallBack():
    def after_calc(self, epoch, val, **kwargs):
        print(f"Done step {epoch}: {val}")
        if val > 10:
            return True

In [68]:
slow_calculation(PrintAfterCallBack())

Done step 0: 0
Done step 1: 1
Done step 2: 5
Done step 3: 14
Stopping early


14

Next, we want to alter the calculation itself:

In [69]:
class SlowCalculator():
    def __init__(self, cb=None):
        self.cb = cb
        self.res = 0
    
    def callback(self, cb_name, *args):
        if not self.cb:
            return
        cb = getattr(self.cb, cb_name, None)
        if cb:
            return cb(self, *args)
    
    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i*i
            sleep(1)
            if self.callback('after_calc', i):
                print("Stopping Early")
                break

In [70]:
class ModifyingCallBack():
    def after_calc(self, calc, epoch):
        print(f"After epoch {epoch}: {calc.res}")
        if calc.res > 10:
            return True
        if calc.res < 3:
            calc.res = calc.res*2

In [74]:
calculator = SlowCalculator(ModifyingCallBack())

In [75]:
calculator.calc()

After epoch 0: 0
After epoch 1: 1
After epoch 2: 6
After epoch 3: 15
Stopping Early


This represent the extend of our callback functionalities that we use in FastAI.

### `__dunder__` thingies

In [76]:
class SloppyAdder():
    def __init__(self, o):
        self.o = o
    
    def __add__(self, b):
        return SloppyAdder(self.o + b.o + 0.01)
    
    def __repr__(self):
        return str(self.o)

In [77]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a + b

3.01

Special Methods I should know:

- [x] __getitem__
- [ ] __getattr__
- [ ] __setattr__
- [ ] __del__
- [x] __init__
- [ ] __new__
- [ ] __enter__
- [ ] __exit__
- [x] __len__
- [x] __repr__
- [ ] __str__

## Browsing Source Code

Learn to do these things in our editor of choice:

- [ ] Jump to tag/symbol by with (with completion)
    - Symbol: class/fucntion/...
- [ ] Jump to current tag
    - By clicking on the tag
- [ ] Jump to library tags
- [ ] Go back
    - To the place you were working in (in the file)
- [ ] Search
- [ ] Outlining/Folding

## Variance & Stuff

### Variance

Variance is the average of how far each data point is from the mean $\mu$.

In [78]:
t = torch.Tensor([1., 2., 4., 18.])

In [79]:
m = t.mean(); m

tensor(6.2500)

In [80]:
(t-m).mean()

tensor(0.)

We can't do that because all of the negative/positive differences cancel-out, so we can fix that in one of two ways:

In [81]:
(t-m).pow(2).mean()

tensor(47.1875)

The **mean absolute deviation**:

In [84]:
# Or..
(t-m).abs().mean()

tensor(5.8750)

Let's undo the squaring that happened for the first solution, we present the **standard deviation**:

In [85]:
(t-m).pow(2).mean().sqrt()

tensor(6.8693)

they're still different, why?

Because of the squaring mechanism, standard deviation is more sensitive to outliers.